In [ ]:
import numpy as np
from scipy import constants 
import matplotlib.pyplot as plt
import os

def fftconvolve(ax,ay,axis=-1):
    nex=ax.shape[axis]+ay.shape[axis]-1
    return np.fft.ifft(np.fft.fft(ax,n=nex,axis=axis)*np.fft.fft(ay,n=nex,axis=axis),axis=axis)

def DBF(gaz,gele,wl,a0,element_pos,we):
    k=np.pi*2/wl
    # thetaxy = np.arctan2(np.tan(thetay),np.tan(thetax))
    # phixy = np.arctan(np.sqrt(np.tan(thetay)**2+np.tan(thetax)**2))
    ar = np.array([np.sin(gele)*np.sin(gaz), np.sin(gele)*np.cos(gaz), np.cos(gele)])
    ar = ar-a0
    # print(ar)
    Exy=np.zeros((*we.shape[:2],*ar.shape[1:]))
    for ie in range(element_pos.shape[0]):
        de = element_pos[ie,:]
        element_phase = np.sum(ar*de[:,np.newaxis,np.newaxis],axis=0)
        # print(element_phase*180/np.pi)|
        xe = we[:,:,ie]
        Exy = Exy + xe[:,:,np.newaxis,np.newaxis]*np.exp(1j*k*element_phase)[np.newaxis,np.newaxis,:,:]
    return Exy

def element_rotate(element_pos,deltadeg):
    # this rotation is clockwise not math degree
    deltadeg=deltadeg*np.pi/180
    Mr = np.array([[np.cos(deltadeg),np.sin(deltadeg),0],[-np.sin(deltadeg),np.cos(deltadeg),0],[0,0,1]])
    return np.transpose(np.matmul(Mr,np.transpose(element_pos)))

### beamforming @ AZ, ELE 

In [ ]:
# beamaz = np.arange(-180,180,5)
# beamele = np.arange(-45,45,3)
# thetaaz, thetaele = np.meshgrid(beamaz, beamele)
# thetaaz = thetaaz * np.pi/180
# thetaele = thetaele * np.pi/180
# thetax = np.arcsin(np.sin(thetaele)*np.sin(thetaaz))*180/np.pi
# thetay = np.arcsin(np.sin(thetaele)*np.cos(thetaaz))*180/np.pi

### beamforming @ $\theta_x$, $\theta_y$ 

In [ ]:
beamx = np.arange(-45,46,1.5) * np.pi/180
beamy = np.arange(-45,46,1.5) * np.pi/180
thetax, thetay = np.meshgrid(beamx, beamy)
thetaele = np.arccos(np.sqrt(1-np.sin(thetax)**2-np.sin(thetay)**2))
thetaaz = np.arctan2(np.sin(thetax),np.sin(thetay))
thetax = thetax*180/np.pi
thetay = thetay*180/np.pi

In [ ]:
os.makedirs('output/',exist_ok=True)

TS = 290        # noise temperature K
npulse = 16    # number of pulses in sequence
nfft = 512      # zero padding in fft
Pt = 100        # peak power    W
G = 100         # Gain
F0 = 3e9       # carrier frequency Hz
wavelength = constants.c/F0     # radar wavelength  m
B=200e6/10
ts = 5e-9       # sample rate   s
Tx = 1e-6       # pulse width
PRF = 10e3     # PRF   Hz
PRT=1/PRF
Va=constants.c*PRF/4/F0
Lx=Tx/ts
chirp = B/Tx
Ra=constants.c/PRF/2    # maximum unambiguous range
# print(Ra)
rref = 3e3-500
rswath = 1000
tref = 2*rref/constants.c

de = wavelength/2
xx,yy = np.meshgrid(np.arange(-3.5,3.6,1),np.arange(-1.5,1.6,1))
element_loc0 = np.array([xx.flatten(),yy.flatten(),np.zeros(xx.size)]).transpose()*de

theta0 = 0 * np.pi/180  # azimuth y->x
phi0 = 0 * np.pi/180    #zenith
a0 = np.array([np.sin(phi0)*np.sin(theta0), np.sin(phi0)*np.cos(theta0), np.cos(phi0)])[:,np.newaxis,np.newaxis]

txp = np.arange(0,Tx,ts)    # discrete time of pulse
# create pulse window
tmpwd = np.hanning(txp.size//10)
wd = np.ones(txp.size)
wd[:tmpwd.size//2] = tmpwd[:tmpwd.size//2]
wd[-tmpwd.size//2:] = tmpwd[-tmpwd.size//2:]
tx=Pt*wd*np.exp((-np.pi*B*txp+np.pi*chirp*txp**2)*1j) # transmit waveform
tf = np.arange(1e-5*ts,(2*rswath/constants.c)//ts*ts+1e-5*ts,ts)  # fast time
fast_time = tf[np.newaxis,:]
tm = np.arange(npulse)*PRT
tm = tm[:,np.newaxis]
Faxe=np.arange(-1/2/ts,1/2/ts,1/ts/tf.size)  # frequency after fft
Faxe = Faxe[np.newaxis,:]

# fft_wd = 'Hanning'
# r_wd = 'Hanning'
# casename = 'r_'+r_wd+'-v_'+fft_wd

In [ ]:
def fire_pulse(element_loct,target_para):
    total_rx = np.zeros((npulse,tf.size,int(element_loct.shape[0])),dtype=tx.dtype)
    for Rtar,Vtar,rcs, azt, elt in target_para:
        tau_0 = 2*Rtar/constants.c # first pulse target time delay
        FD = 2*Vtar/wavelength  # target doppler
        ARtar = constants.c*(tau_0-np.floor(tau_0/PRT)*PRT)/2 # apperent range of target
        Radar_c = np.sqrt(G**2*wavelength**2*rcs/(4*np.pi)**3)
        # *np.exp(2j*np.pi*np.random.random(1)) # radar constant without scale of range
        rr = constants.c*(tau_0)/2       # range
        Radar_c = Radar_c/rr**2     # return power
    
        rx=np.zeros(tf.shape,dtype=tx.dtype)       # received signal
        rx[:tx.size]=tx
    
        tau_m = 2*(Rtar-tm*Vtar)/constants.c-tref
        rx=np.tile(rx,(npulse,1))
        Radar_c=Radar_c
        frx=np.fft.fftshift(np.fft.fft(rx,axis=1),axes=1) 
            # frequency content of received signal
        dfrx=frx*np.exp(-2j*np.pi*Faxe*tau_m)   # time delay in frequency domain
    
        rx=np.fft.ifft(np.fft.fftshift(dfrx,axes=1))*Radar_c
        rx=rx*np.exp(2j*np.pi*FD*fast_time)*np.exp(-2j*np.pi*F0*tau_m)
        az = azt*np.pi/180
        ele = elt*np.pi/180
        targetRvector = np.array([np.sin(ele)*np.sin(az),np.sin(ele)*np.cos(az),np.cos(ele)])[np.newaxis,:]
        dl = np.sum(element_loct*targetRvector,axis=1)[np.newaxis,np.newaxis,:]
        total_rx=total_rx+rx[:,:,np.newaxis]*np.exp(-2j*np.pi*dl/wavelength)
    return total_rx

In [ ]:
def all_digital(element_loct,rx):
    matchfilter=np.flipud(np.conj(tx))[np.newaxis,:,np.newaxis]
    crx = fftconvolve(rx,matchfilter,1)[:,-rx.shape[1]:,:]
    Eloc = DBF(thetaaz,thetaele,wavelength,a0,element_loct,crx)
    Beam = np.mean(np.abs(Eloc)**2,axis=0)
    r=constants.c*tf/2+rref
    plt.pcolormesh(np.squeeze(thetax),np.squeeze(thetay),np.squeeze(Beam[np.abs(r-3e3)==np.min(np.abs(r-3e3)),:,:]))
    plt.ylim(-40,40)
    plt.xlim(-40,40)
    plt.colorbar()

In [ ]:
def subarray(element_loct,rx):
    subarray_rx = np.sum(rx.reshape((rx.shape[0],rx.shape[1],4,8)), axis=2)
    subarray_loc0 = np.mean(element_loct.reshape((4,8,3)), axis=0)
    matchfilter=np.flipud(np.conj(tx))[np.newaxis,:,np.newaxis]
    crx = fftconvolve(subarray_rx,matchfilter,1)[:,-rx.shape[1]:,:]
    Eloc = DBF(thetaaz,thetaele,wavelength,a0,subarray_loc0,crx)
    Beam = np.mean(np.abs(Eloc)**2,axis=0)
    # Beam = np.log10(Beam/np.max(Beam))
    r=constants.c*tf/2+rref
    # plt.pcolormesh(np.squeeze(thetax),np.squeeze(thetay),np.squeeze(Beam[np.abs(r-3e3)==np.min(np.abs(r-3e3)),:,:]),vmin=-3,vmax=0)
    plt.pcolormesh(np.squeeze(thetax),np.squeeze(thetay),np.squeeze(Beam[np.abs(r-3e3)==np.min(np.abs(r-3e3)),:,:]))
    plt.ylim(-40,40)
    plt.xlim(-40,40)
    plt.colorbar()

## set target

In [ ]:
# Rtar_list = [2.95e3,3e3,3.05e3,3.05e3]        # target range

# Rtar_list = [3e3,3e3,3e3,3e3]        # target range
# Atar_list = [90,     90,  90, 90]              # target azimuth
# Etar_list = [0,     0,  30,-10]              # target elevation
# Vtar_list = [-20,0,15,-30]               # fix target radial velocity
# rcs_list = [1000,1000,1000,1000]         # radar cross section   m^2

Rtar_list = [3e3,3e3]        # target range
Atar_list = [ 90,0]              # target azimuth
Etar_list = [-30,20]              # target elevation
Vtar_list = [-30,0]               # fix target radial velocity
rcs_list = [1000,1000]         # radar cross section   m^2


## array orientation 0 (y+)

In [ ]:
baseline_azimuth = 0
element_loct = element_rotate(element_loc0,baseline_azimuth)
target_para = zip(Rtar_list, Vtar_list, rcs_list, Atar_list, Etar_list)
rx = fire_pulse(element_loct,target_para)

### All digital beam forming

In [ ]:
all_digital(element_loct,rx)
plt.plot(np.array([-90,90])*np.cos(baseline_azimuth*np.pi/180),np.array([-90,90])*np.sin(baseline_azimuth*np.pi/180),'r')
plt.axis([-40,40,-40,40])

### Subarray beam forming

In [ ]:
subarray(element_loct,rx)

In [ ]:
baseline_azimuth = 30
element_loct = element_rotate(element_loc0,baseline_azimuth)
target_para = zip(Rtar_list, Vtar_list, rcs_list, Atar_list, Etar_list)
rx = fire_pulse(element_loct,target_para)
plt.figure()
all_digital(element_loct,rx)
plt.plot(np.array([-90,90])*np.cos(baseline_azimuth*np.pi/180),np.array([-90,90])*np.sin(baseline_azimuth*np.pi/180),'r')
plt.axis([-40,40,-40,40])
plt.figure()
subarray(element_loct,rx)

In [ ]:
baseline_azimuth = 120
element_loct = element_rotate(element_loc0,baseline_azimuth)
target_para = zip(Rtar_list, Vtar_list, rcs_list, Atar_list, Etar_list)
rx = fire_pulse(element_loct,target_para)
plt.figure()
all_digital(element_loct,rx)
plt.plot(np.array([-90,90])*np.cos(baseline_azimuth*np.pi/180),np.array([-90,90])*np.sin(baseline_azimuth*np.pi/180),'r')
plt.axis([-40,40,-40,40])
plt.figure()
subarray(element_loct,rx)